In [1]:
import pandas as pd
pd.options.plotting.backend = "plotly"
from datetime import datetime, date
import pylatex as pl
from pylatex.utils import NoEscape
import pymc as pm
import arviz as az
import nevergrad as ng
import plotly.express as px

from estival.model import BayesianCompartmentalModel
from estival.optimization.nevergrad import optimize_model
from estival.priors import UniformPrior
from estival.targets import NegativeBinomialTarget, CustomTarget
from estival.calibration import pymc as epm
from tbdynamics import model
from tbdynamics.inputs import *
from tbdynamics.utils import build_contact_matrix

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
time_start = 1800
time_end = 2020

In [3]:
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
age_strata = [0,5,15,35,50]

In [4]:
params = {
    "contact_rate": 0.009414102898074345,
    "start_population_size": 280000,
    "cdr_adjustment": 0.6,
    "progression_multiplier": 1.1,
    "infectious_seed": 1,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
    "infect_death_rate_unstratified": 0.21,
    "on_treatment_infect_multiplier": 0.08,
    'smear_positive_death_rate':0.364337776897486,
    'smear_negative_death_rate': 0.027588310343242016, 
    'smear_positive_self_recovery': 0.20344728302826143,
    'smear_negative_self_recovery': 0.22723824998716693,
}

In [5]:
tb_model, build_text = model.build_base_model(compartments, infectious_compartments, time_start, time_end)
build_text

"The base model consists of 6 states, representing the following states: susceptible, early_latent, late_latent, infectious, on_treatment, recovered. Only the ['infectious', 'on_treatment'] compartment contributes to the force of infection. The model is run from 1800 to 2020. "

In [6]:
model.set_starting_conditions(tb_model)


'The simulation starts with Parameter start_population_size million fully susceptible persons, with infectious persons introduced later through strain seeding as described below. '

In [7]:
model.add_entry_flow(tb_model)

'The birth process add newborns to the susceptible compartment of the model'

In [8]:
model.add_natural_death_flow(tb_model)

'The universal_death process add universal death to the model'

In [9]:
model.add_infection(tb_model)

('The infection process moves people from the susceptible compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The infection_from_latent process moves people from the late_latent compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The infection_from_recovered process moves people from the recovered compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ')

In [10]:
model.add_latency(tb_model)

('The stabilisation process moves people from the early_latent compartment to the late_latent compartment, under the frequency-dependent transmission assumption. ',
 'The early_activation process moves people from the early_latent compartment to the infectious compartment, under the frequency-dependent transmission assumption. ',
 'The late_activation process moves people from the late_latent compartment to the infectious compartment, under the frequency-dependent transmission assumption. ')

In [11]:
model.add_infect_death(tb_model)

'The infect_death process moves people from the infectious'

In [12]:
model.add_self_recovery(tb_model)

'The self_recovery process moves people from the on_treatment compartment to the recovered, under the frequency-dependent transmission assumption. '

In [13]:
model.add_detection(tb_model)

'The detection process moves people from the infectious compartment to the on_treatment compartment, under the frequency-dependent transmission assumption. '

In [14]:
model.add_treatment_related_outcomes(tb_model)

('The treatment_recovery process moves people from the on_treatment compartment to the recovered compartment, under the frequency-dependent transmission assumption. ',
 'The treatment_death process moves people from the on_treatment compartment to the death, under the frequency-dependent transmission assumption. ',
 'The early_activation process moves people from the on_treatment compartment to the infectious compartment, under the frequency-dependent transmission assumption. ')

In [15]:
matrix = build_contact_matrix()

In [16]:
age_strat, des = model.add_age_strat(compartments, infectious_compartments, age_strata, matrix, fixed_parameters)
des

"The age stratification adjusts following process: (1) The universal death by age group. The data was taken from autumn's database. (2) The early and late activation(3) Age infectioness switched at age of 15(4) Infectiousness multiplier for treatment(5) Treatment outcomes: relapse, recovery and death"

In [17]:
tb_model.stratify_with(age_strat)

In [20]:
 organ, des = model.add_organ_strat(fixed_parameters,infectious_compartments)
des

{'smear_positive': <Overwrite 'Parameter smear_positive_death_rate'>, 'smear_negative': <Overwrite 'Parameter smear_negative_death_rate'>, 'extrapulmonary': <Overwrite 'Parameter smear_negative_death_rate'>}


Stratification: organ

In [21]:
tb_model.stratify_with(organ)

AttributeError: 'str' object has no attribute '_validate'

In [ ]:
model.request_output(tb_model, compartments,  latent_compartments, infectious_compartments)

In [ ]:
priors = [
    UniformPrior("start_population_size", (150000, 300000)),
    UniformPrior("contact_rate", (0.0001, 0.02)),
    #UniformPrior("infectious_seed", [100, 2000]),
    UniformPrior("rr_infection_latent", (0.2, 0.5)),
    UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),
    UniformPrior("cdr_adjustment", (0.6, 1.0)),
    # UniformPrior("progression_multiplier", (0.1, 2.0)),
    # UniformPrior("cdr_adjustment", [0.6, 1.0]),
    # UniformPrior("infect_death_rate_dict.smear_positive", [0.335, 0.449]),
    # UniformPrior("infect_death_rate_dict.smear_negative", [0.017, 0.035]),
    # UniformPrior("self_recovery_rate_dict.smear_positive", [0.177, 0.288]),
    # UniformPrior("self_recovery_rate_dict.smear_negative", [0.073, 0.209]),
    # UniformPrior("rr_progression_diabetes", [1, 10]),
]
pop = pd.Series({2009: 1207100, 2019: 1194300})
cases_intindex = pop.copy()
cases_intindex.index = pop.index - time_start

def least_squares(modelled, obs, parameterss, time_weights):
    return 0.0 - (((modelled - obs) ** 2.0)).sum()
targets = [
    CustomTarget("total_population", cases_intindex, least_squares),
]
binom_targets = [
    NegativeBinomialTarget("total_population", cases_intindex, 20.0),
]
calibration_model = BayesianCompartmentalModel(tb_model, params, priors, binom_targets)

In [ ]:
tb_model.run(params)

In [ ]:
tb_model.get_derived_outputs_df()

In [ ]:
optimise_model = True
if optimise_model:
    print("Optimising with nevergrad \n Progression of loss function values:")
    optim_runner = optimize_model(calibration_model)
    for i in range(10):
        rec = optim_runner.minimize(100)
        print(rec.loss)
    optim_params = rec.value[1]
    params.update(optim_params)
    tb_model.run(parameters=params)
    print("Best calibration parameters found:")
    print(optim_params)